In [1]:
import ee
from osgeo import ogr
import geopandas as gpd
import geemap
from geemap import ml
import pickle


ee.Initialize()

In [3]:
# USE THIS CHUNK TO CHECK OPERATION STATUS IF YOU QUITTED NOTEBOOK

#ee.data.listOperations()

[{'name': 'projects/earthengine-legacy/operations/RJ6LYQGGGJSOHSR5ZGFLBG5A',
  'metadata': {'@type': 'type.googleapis.com/google.earthengine.v1alpha.OperationMetadata',
   'state': 'SUCCEEDED',
   'description': 'Lazio_2018',
   'createTime': '2022-06-01T13:19:33.006429Z',
   'updateTime': '2022-06-01T14:57:05.880694Z',
   'startTime': '2022-06-01T13:19:40.365644Z',
   'endTime': '2022-06-01T14:57:05.880694Z',
   'type': 'EXPORT_IMAGE',
   'destinationUris': ['https://drive.google.com/#folders/1ylCKycX24ELqv1RU9_clkAFSEQ4wflu9'],
   'attempt': 1,
   'progress': 1,
   'stages': [{'displayName': 'Create Local Files',
     'completeWorkUnits': 1,
     'totalWorkUnits': '1',
     'description': 'Computation and writing of temporary files.'},
    {'displayName': 'Write Files to Destination',
     'completeWorkUnits': 1,
     'totalWorkUnits': '1',
     'description': 'Uploading of files to the export destination.'}],
   'batchEecuUsageSeconds': 237112.59375},
  'done': True,
  'response': {

In [2]:
# SOME REFERENCES

# https://courses.spatialthoughts.com/end-to-end-gee.html
# https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/
# https://developers.google.com/earth-engine/guides/machine-learning
# https://geemap.org/notebooks/46_local_rf_training/#train-a-model-locally-using-scikit-learn

In [11]:
path = 'C:/Users/drikb/Desktop/Tirocinio/EarthEngine/'
geobounds = gpd.read_file(path + 'Lazio_polygon.shp')

In [35]:
lazio_shape = geobounds['geometry'].iloc[0]
lazio_list = []
for polygon in lazio_shape.geoms:
    lazio_list.append([[x, y] for x, y in zip(polygon.exterior.xy[0], polygon.exterior.xy[1])])

In [36]:
poly = ee.Geometry.MultiPolygon(lazio_list, 'EPSG:4326')

In [37]:
start_date = '2018-01-01'
end_date = '2018-12-31'
lowresbands = ['B5', 'B6', 'B7', 'B8A', 'B1', 'B9', 'B11', 'B12']
bands = ['B4', 'B3', 'B2', 'B8', 'B5', 'B6', 'B7', 'B8A', 'B1', 'B9', 'B11', 'B12']

In [38]:
lazio_2018_coll = (ee.ImageCollection('COPERNICUS/S2_SR')
                  .filterBounds(poly)
                  .filterDate(start_date, end_date)
                  .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 30)))

In [39]:
projection = lazio_2018_coll.first().select('B4').projection()

In [40]:
lazio_2018 = lazio_2018_coll.select(bands).reduce(ee.Reducer.median())

In [41]:
lazio_2018 = lazio_2018.clip(poly)

In [ ]:
lazio_2018 = lazio_2018.uint16()

In [42]:
Map = geemap.Map()

In [43]:
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 3000
}

In [44]:
Map.addLayer(lazio_2018, vizParams, 'Median Map')

In [45]:
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [47]:
task = ee.batch.Export.image.toDrive(lazio_2018,
                                    description = 'Lazio_2018',
                                     folder = 'EE_images',
                                     fileFormat = 'GeoTIFF',
                                     crs = projection.crs(),
                                     scale = 10,
                                    maxPixels = 62314055196)

In [48]:
task.start()

In [50]:
task.status()

{'state': 'RUNNING',
 'description': 'Lazio_2018',
 'creation_timestamp_ms': 1654089573006,
 'update_timestamp_ms': 1654091026794,
 'start_timestamp_ms': 1654089580365,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'RJ6LYQGGGJSOHSR5ZGFLBG5A',
 'name': 'projects/earthengine-legacy/operations/RJ6LYQGGGJSOHSR5ZGFLBG5A'}